In [112]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter

from langchain.chat_models import ChatOpenAI
import os

In [113]:
os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"

In [116]:
import requests
import pandas as pd
import validators


apikey = "YOUR-API-KEY"

def get_jsonparsed_data(ticker, quarter, year):

    url = f"https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?quarter={quarter}&year={year}&apikey={apikey}"

    if validators.url(url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Unable to fetch data, please check the url"}
    else:
        return {"error": "Invalid url"}

In [118]:
quarters = [1, 2, 3, 4]
years = [2021, 2022, 2023]
ticker = "TSLA"
transcript_summary = ""

for year in years:
    for quarter in quarters:
        data = get_jsonparsed_data(ticker, quarter, year)
        if "error" in data:
            print(data["error"])
        else:
            if len(data) > 0:
                transcript_summary += data[0]["content"]


In [119]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.create_documents([transcript_summary])

In [114]:
# store
store = LocalFileStore("./cache/")

In [115]:
# instantiate embedding model
embeddings_model = OpenAIEmbeddings()

# pass in our vector store
embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model,
    store
)

In [121]:
# create vector store, we use FAISS in this case
vector_store = FAISS.from_documents(chunks, embedder)

In [122]:
# this is the entire retrieval system
medium_qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    verbose=True
)

In [131]:
prompts = ["Why did the company’s revenue and income increase or decline?", 
            "What are the challenges that should be anticipated?",
            "Where and why are the sales trending?",
            "Who are the emerging competitors in the industry?"]

In [132]:
def process_prompt(prompt):
    print(f"{prompt}\n")
    print("------------------------------------")
    print("Default OpenAI Response")
    print("------------------------------------\n")
    client = openai.Client()
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=500)
    print(f"{response.choices[0].text.strip()}\n")
    
    print("------------------------------------")
    print("RAG Response")
    print("------------------------------------\n")
    response_rag = medium_qa_chain({"query":prompt})
    print(f"{response_rag['result']}\n")

for prompt in prompts:
    process_prompt(prompt)


Why did the company’s revenue and income increase or decline?

------------------------------------
Default OpenAI Response
------------------------------------

There are several possible reasons why a company's revenue and income may have increased or declined, including:

1. Changes in demand for the company's products or services: If there is an increase in demand for the company's offerings, it is likely to lead to an increase in revenue and income. Conversely, a decline in demand can result in a decrease in revenue and income.

2. Changes in pricing strategies: If the company increases or decreases its prices, it can impact its revenue and income. For example, if the company decreases its prices, it may lead to an increase in sales and higher revenue, but could also result in a decrease in profits.

3. Economic conditions: Changes in the overall economy can affect a company's revenue and income. For instance, during a recession, consumers may be more cautious with their spending,